In [1]:
import pandas as pd 
import numpy as np 
import networkx as nx
import scipy.special 
import pandas_datareader as pdr 
from matplotlib import pyplot as plt
from datetime import date, datetime
import math
import random
from plotly.offline import iplot
import plotly.graph_objs as go

from tools.comparisons import *
from tools.data_transform import *
from hypotheses.sign_statistics import *
from hypotheses.sign_tests import *


/Users/stasyan/PycharmProjects/SignProcedure/venv/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


Проверка статистик для гипотезы о трех акциях

In [13]:
corr = [[1, 0, 0.2], \
        [0, 1, 0], \
        [0.2, 0, 1]]
new_rets = get_norm_seq(corr, 5)
new_inds = get_inds_matrix(new_rets)
new_inds

[[0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 1]]

In [14]:
print('Кол-во дней, когда i = j != k : ',stat_T2(new_inds, 0, 1, 2))
print('Кол-во дней, когда i = k != j : ',stat_T3(new_inds, 0, 1, 2))

Кол-во дней, когда i = j != k :  0
Кол-во дней, когда i = k != j :  2


Верно

Проверка статистик для гипотезы о четырех акциях

In [16]:
corr = [[1, 0, 0.3, 0.2], \
        [0, 1, 0, 0.2], \
        [0.3, 0, 1, 0], \
        [0.2, 0.2, 0, 1]]
new_rets = get_norm_seq(corr, 5)
new_inds = get_inds_matrix(new_rets)
new_inds

[[1, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]

In [17]:
print('Кол-во дней, когда i = j и k != l : ',stat_R2(new_inds, 0, 1, 2, 3))
print('Кол-во дней, когда i != j и k = l : ',stat_R3(new_inds, 0, 1, 2, 3))

Кол-во дней, когда i = j и k != l :  3
Кол-во дней, когда i != j и k = l :  2


Верно

##### Функция подсчета порога c для теста 

Вопрос:    

Допустим, у нас акция i сильно коррелирует с k, а акция j сильно коррелирует с l (это эль), поэтому если знак i совпадает со знаком j в день t, то и знак k с большой вероятностью совпадет со знаком l в день t. И наоборот, если i не совпадет с j в день t, то и k не совпадет с l в день t с большой вероятностью. То есть таких случаев из выборки будет большинство.        

Допустим, наблюдений немного и случаев, что (i = j, a k != l) или (i != j, a k = l) выпало всего 3 (допустим). А alpha мы взяли 0.1.     

То есть по сути, мы получаем число $k = k_2 + k_3 = 3$,  $alpha = 0.1$, тогда нет такой нижней границы $c$, чтобы выполнилось неравенство $􏰋\sum_{i=c}^k С_k^i * (1/2)^k \leq  alpha$ 


Или более вероятный вариант:    

Гипотеза $H_{ijk}$ о трех акция. Если j и k будут сильно коррелировать и изменяться в одном направлении, то дней, когда их знаки не совпадут, будет очень мало, а это и есть $k = k_2 + k_3$ будет мало. Даже если выборка большая. 


Вопрос:  

Если у нас большая выборка, которую мы нагенерировали для сравнения, например $n = 10000$. То если у нас выпадет $k = k_2 + k_3 = 1100$ (например) или больше, то биномиальный коэффициент (например, 500 из 1100) = inf. 

Проверка

In [20]:
print(get_threshold_c2(0.1, 3))
print(get_threshold_c1(0.1, 3))

4

Вот этот случай, описанный выше

In [22]:
print(get_threshold_c2(0.3, 7))
print(get_threshold_c1(0.3, 7))

5

In [24]:
print(get_threshold_c2(0.3, 8))
print(get_threshold_c1(0.3, 8))

6

##### Функция теста для трех акций, ret_inds - индикаторы доходностей

In [29]:
# k = k2 + k3
c = get_threshold_c2(0.6, 100)
k2 = 50
print(c)
if k2 == c:
  print('hey')
else:
  print('fg')

50
hey


### Блок проверки тестов 

##### Для гипотезы о трех акциях: $h_{ijk}: \gamma_{ij}^{Sg} \leq \gamma_{ik}^{Sg}$



In [0]:
stcks= ['i', 'j', 'k']
corr = [[1, 0, 0.2], \
        [0, 1, 0], \
        [0.2, 0, 1]]

Проверим, правильно ли генерируются новые СВ по заданной матрице корреляций. Для этого сгенерируем СВ и посчитаем для них матрицу корреляции, она должна быть приближенна к истинной. 

In [39]:
new_rets = get_norm_seq(corr, 100)
new_corr = np.corrcoef(new_rets)
new_corr

array([[ 1.        ,  0.16056888,  0.25568921],
       [ 0.16056888,  1.        , -0.00627328],
       [ 0.25568921, -0.00627328,  1.        ]])

Такое себе, увеличим n

In [40]:
new_rets = get_norm_seq(corr, 1000)
new_corr = np.corrcoef(new_rets)
new_corr

array([[ 1.        , -0.00791548,  0.20453167],
       [-0.00791548,  1.        , -0.04590673],
       [ 0.20453167, -0.04590673,  1.        ]])

Поближе, увеличим еще n

In [41]:
new_rets = get_norm_seq(corr, 10000)
new_corr = np.corrcoef(new_rets)
new_corr

array([[ 1.00000000e+00,  4.96236071e-04,  2.00311359e-01],
       [ 4.96236071e-04,  1.00000000e+00, -1.16423283e-02],
       [ 2.00311359e-01, -1.16423283e-02,  1.00000000e+00]])

Ок, вроде новые СВ по заданной матрице считаются правильно, и чем больше n, тем ближе матрица корреляций по новым СВ к истинной матрице корреляций. Хотя при n=100 не очень хорошее приближение, но нам по сути это пока не нужно

Проверим, правильно ли считаются индикаторы доходностей


In [42]:
new_rets = get_norm_seq(corr, 5)
new_rets

array([[ 1.07662488,  0.3215481 ,  0.66074523,  1.22201804, -0.67726158],
       [ 1.14246472, -2.0274276 , -0.47737966,  0.55464585,  1.2838995 ],
       [ 0.91512288,  0.6680056 ,  0.32166698,  0.67945834, -0.29632213]])

In [43]:
new_inds = get_inds_matrix(new_rets)
new_inds

[[1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]

ОК

Теперь проверяем сам тест, для этого 1000 раз генерируем новые последовательности СВ по нашей истиной матрице корреляций (в которой корреляция между i и j = 0 и она меньше, чем корреляция между i и k = 0.2. Уровень значимости $\alpha = 0.1$

In [5]:
stcks= ['i', 'j', 'k']
corr = [[1, 0, 0], \
        [0, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=10, title='Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')


stcks= ['i', 'j', 'k', 'l']
corr = [[1, 0, 0, 0], \
        [0, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=10, title='Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0, 0]
[0, 1, 0]
[0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]


In [10]:
for _ in range(1):
    stcks= ['i', 'j', 'k']
    corr = [[1, 0.2, 0.2], \
            [0.2, 1, 0.2], \
            [0.2, 0.2, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Ошибки первого рода', \
               title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

    stcks= ['i', 'j', 'k', 'l']
    corr = [[1, 0, 0, 0], \
            [0, 1, 0, 0], \
            [0, 0, 1, 0.2], \
            [0, 0, 0.2, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Ошибки первого рода', \
               title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.2, 0.2]
[0.2, 1, 0.2]
[0.2, 0.2, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0.2]
[0, 0, 0.2, 1]


In [8]:
for _ in range(10):
    stcks= ['i', 'j', 'k']

    corr = [[1, 0.2, 0], \
            [0.2, 1, 0], \
            [0, 0, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
               title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

    corr = [[1, 0.5, 0], \
            [0.5, 1, 0], \
            [0, 0, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
               title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')



    stcks= ['i', 'j', 'k', 'l']

    corr = [[1, 0.2, 0, 0], \
            [0.2, 1, 0, 0], \
            [0, 0, 1, 0], \
            [0, 0, 0, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
               title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

    corr = [[1, 0.5, 0, 0], \
            [0.5, 1, 0, 0], \
            [0, 0, 1, 0], \
            [0, 0, 0, 1]]

    test_rates(corr, stcks, 'simple', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
               title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.2, 0]
[0.2, 1, 0]
[0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.5, 0]
[0.5, 1, 0]
[0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.2, 0, 0]
[0.2, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.5, 0, 0]
[0.5, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.2, 0]
[0.2, 1, 0]
[0, 0, 1]


...

alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.5, 0, 0]
[0.5, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.2, 0]
[0.2, 1, 0]
[0, 0, 1]


alpha =  0.1
alpha =  0.05
alpha =  0.01
Correlations matrix:
[1, 0.5, 0]
[0.5, 1, 0]
[0, 0, 1]


alpha =  0.1
alpha =  0.05


KeyboardInterrupt: 

In [0]:
# stcks= ['i', 'j', 'k']
# corr = [[1, 0.5, 0], \
#         [0.5, 1, 0.5], \
#         [0, 0.5, 1]]

# test_rates(corr, stcks, alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Ошибки первого рода', \
#            title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')


# stcks= ['i', 'j', 'k', 'l']
# corr = [[1, 0.5, 0, 0], \
#         [0.5, 1, 0.5, 0], \
#         [0, 0.5, 1, 0.5], \
#         [0, 0, 0.5, 1]]

# test_rates(corr, stcks, alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Ошибки первого рода', \
#            title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

In [0]:
# stcks= ['i', 'j', 'k']
# corr = [[1, 0.5, 0], \
#         [0.5, 1, 0.5], \
#         [0, 0.5, 1]]

# test_rates(corr, stcks, alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
#            title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')



# stcks= ['i', 'j', 'k', 'l']
# corr = [[1, 0.5, 0, 0], \
#         [0.5, 1, 0.5, 0], \
#         [0, 0.5, 1, 0.5], \
#         [0, 0, 0.5, 1]]

# test_rates(corr, stcks, alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Мощность теста', \
#            title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

Для новых сложных гипотез: complex rand test

In [89]:
stcks= ['i', 'j', 'k']
corr = [[1, 0, 0], \
        [0, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

corr = [[1, 0.2, 0], \
        [0.2, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

corr = [[1, 0.5, 0], \
        [0.5, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')



stcks= ['i', 'j', 'k', 'l']
corr = [[1, 0, 0, 0], \
        [0, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

corr = [[1, 0.2, 0, 0], \
        [0.2, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

corr = [[1, 0.5, 0, 0], \
        [0.5, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложная тест со случайным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


In [18]:
stcks= ['i', 'j', 'k']
corr = [[1, 0, 0], \
        [0, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

corr = [[1, 0.2, 0], \
        [0.2, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

corr = [[1, 0.5, 0], \
        [0.5, 1, 0], \
        [0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')



stcks= ['i', 'j', 'k', 'l']
corr = [[1, 0, 0, 0], \
        [0, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Ошибки первого рода', \
           title_x='кол-во наблюдений' , title_y='доля ошибок первого рода из 1000')

corr = [[1, 0.2, 0, 0], \
        [0.2, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

corr = [[1, 0.5, 0, 0], \
        [0.5, 1, 0, 0], \
        [0, 0, 1, 0], \
        [0, 0, 0, 1]]

test_rates(corr, stcks, 'complex_rand', alpha_list=[0.1, 0.05, 0.01], iters=1000, title='Сложный тест с максимальным выбором: Мощность теста', \
           title_x='кол-во наблюдений' , title_y='мощность теста (при 1000 итерациях)')

alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01


alpha =  0.1
alpha =  0.05
alpha =  0.01
